# Building Asynchronous Gemini Pipelines with **GenAI Processors**

**Release date:** 10 Jul 2025  
**Library version used:** `genai-processors 1.0.3`  

> *This tutorial notebook is designed for absolute beginners. Each section mixes explanation with runnable code so you can learn by doing.*

## Table of Contents
1. [Prerequisites](#Prerequisites)
2. [Installation](#Installation)
3. [Quickstart](#Quickstart)
4. [Core Concepts](#Core-Concepts)
5. [Hands‑On Examples](#Hands-On-Examples)
6. [Testing Your Processors](#Testing-Your-Processors)
7. [Performance Tips](#Performance-Tips)
8. [Deployment Snippet](#Deployment-Snippet)
9. [Next Steps & Exercises](#Next-Steps-&-Exercises)
10. [Resources](#Resources)


## 1. Prerequisites
- **Python ≥ 3.10**
- A **Google Cloud** project with the **Gemini API** enabled (for live examples)
- Basic familiarity with running cells in Jupyter
- A virtual environment is strongly recommended.

## 2. Installation
Run the cell below to install the library. The `--upgrade` flag is a good habit to make sure you get the latest patch version.

In [ ]:
!pip install --upgrade pip wheel
!pip install genai-processors

## 3. Quickstart<a id='Quickstart'></a>
The next cell creates the simplest possible pipeline: text in → Gemini model → text out. The call is **fully async**, so we wrap it in an `asyncio` event loop.

In [ ]:
import asyncio
from genai_processors.core import genai_model
from genai_processors import content_api

async def hello_gemini():
    pipeline = genai_model.GenaiModel(model_name='gemini-2.0-flash')
    inp = content_api.from_text('Hello, GenAI Processors!')
    async for part in pipeline(inp):
        if part.text:
            print(part.text)

await hello_gemini()

## 4. Core Concepts<a id='Core-Concepts'></a>

### 4.1 Processor & PartProcessor
`Processor` is an **async function or class** that receives a *stream* of `ProcessorPart` objects and emits a (possibly transformed) stream.  
Use the `@processor.PartProcessor` decorator for quick one‑off functions.

### 4.2 ProcessorPart
A `ProcessorPart` wraps **content + metadata** (e.g.\ MIME type, role, custom attributes).  
It can represent text, images, audio frames, or arbitrary JSON.

### 4.3 Chaining & Parallelism
- Use **`+`** to chain processors sequentially (output of left → input of right).  
- Use **`//`** to run processors in *parallel* on the same input stream.

The library schedules downstream work as soon as a **single part** is ready, which drastically cuts **Time‑To‑First‑Token** compared with fully synchronous pipelines.

## 5. Hands‑On Examples<a id='Hands-On-Examples'></a>

### 5.1 Example 1 – Uppercase → Gemini

In [ ]:
from genai_processors import processor
from genai_processors.core import genai_model, streams, content_api

@processor.PartProcessor
async def uppercase(part):
    if part.text:
        part.text = part.text.upper()
    return part

async def run_pipeline():
    text_pipeline = uppercase + genai_model.GenaiModel(model_name='gemini-2.0-pro')
    inp = content_api.from_text('Synchronous code is so 2024.')
    async for part in text_pipeline(inp):
        if part.text:
            print(part.text)

await run_pipeline()

### 5.2 Example 2 – Live Audio & Video Agent *(optional)*
This example needs microphone/camera access and valid Gemini Live API credentials, so feel free to skip in a binder environment.

In [ ]:
# Uncomment and configure credentials first
# from genai_processors.core import audio_io, video, live_model, streams
# cam_mic   = video.VideoIn() + audio_io.PyAudioIn()
# speaker   = audio_io.PyAudioOut()
# live_llm  = live_model.LiveProcessor(model_name='gemini-2.0-pro')
# live_agent = cam_mic + live_llm + speaker
# await live_agent(streams.endless_stream())

### 5.3 Example 3 – Emitting Custom JSON Parts

In [ ]:
from genai_processors import processor, content_api, streams

@processor.PartProcessor
async def sentiment_json(part):
    if part.text:
        score = 1 if '🙂' in part.text else -1
        yield content_api.from_json({'sentiment': score})

async def demo_json():
    pipe = sentiment_json
    inp = content_api.from_text('I love async programming 🙂')
    async for part in pipe(inp):
        print(part.json())

await demo_json()

## 6. Testing Your Processors<a id='Testing-Your-Processors'></a>

In [ ]:
# If running `pytest`, save this as test_uppercase.py
import pytest, asyncio
from genai_processors import content_api, streams
from tutorial_notebook import uppercase

@pytest.mark.asyncio
async def test_uppercase():
    s = streams.stream_content(['abc'])
    out = [p async for p in uppercase(s)]
    assert out[0].text == 'ABC'

## 7. Performance Tips<a id='Performance-Tips'></a>
- **Chunk large inputs:** emit smaller `ProcessorPart`s so downstream stages start sooner.
- **Avoid global state:** processors should be pure; easier to parallelise.
- **Use `LiveProcessor`** for streaming token‑level output.
- **Profile:** `asyncio.run(main(), debug=True)` activates slow‑task logging.

## 8. Deployment Snippet<a id='Deployment-Snippet'></a>

In [ ]:
%%bash
cat <<'DOCKER' > Dockerfile
FROM python:3.12-slim
WORKDIR /app
COPY . /app
RUN pip install --no-cache-dir genai-processors uvicorn
CMD ["python", "-m", "uvicorn", "app:router", "--host=0.0.0.0", "--port=8080"]
DOCKER

## 9. Next Steps & Exercises<a id='Next-Steps-&-Exercises'></a>
1. **Write a LowercaseTextProcessor** mirroring the uppercase example.
2. **Parallel audio transcription + sentiment analysis:** Use `//` to fan‑out.
3. **Error handling challenge:** Make a processor that retries Gemini calls with exponential backoff.

## 10. Resources<a id='Resources'></a>

| Resource | Link |
|----------|------|
| Blog Announcement | https://developers.googleblog.com/en/genai-processors/ |
| GitHub Repo | https://github.com/google-gemini/genai-processors |
| Live API Docs | https://ai.google.dev/gemini-api/docs/live |
| AsyncIO Primer | https://medium.com/%40Shrishml/all-about-python-asyncio-ca1f5a8974b0 |
| uvloop & async performance | https://ai.plainenglish.io/the-role-of-uvloop-in-async-python-for-ai-and-machine-learning-pipelines-c7fec45a4966 |
| OpenTools AI overview | https://opentools.ai/news/google-deepmind-launches-genai-processors-revolutionizing-ai-development |
